# Analyzing `GET /hub/api/users?state=...`

for validating [this jupyterhub PR](https://github.com/jupyterhub/jupyterhub/pull/3177)


These runs were collected on GCP `e2-standard-32` VMs, with jupyterhub 1.2.1 and PR 3177 merged into master,
run with both postgres and sqlite, since this involves database queries.

In [1]:
import os
import glob
import pandas as pd
import altair as alt

## Loading the data

Our data is json-formatted log messages produced by python-json-logger.
Only a subset of these contain  sample measurmements, tagged with `test="user_list_test"`.

In [2]:
path = "runs/after-sqlite/05000-01000.jsonl"
df = pd.read_json(path, lines=True)
df

,message,endpoint,token,dry_run,log_to_file,verbose,command,keep,count,batch_size,total_count,state,time,found,total,active,test
0,"Args: {""batch_size"": 20, ""command"": ""user-stre...",http://127.0.0.1:8000/hub/api,***,0.0,05000-01000.jsonl,0.0,user-stress-test,1.0,1000.0,20.0,5000.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Took 0.211 seconds to find_existing_stress_tes...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Have 5000/5000, creating 0 more",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Starting 1000 notebook servers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Took 75.807 seconds to start_servers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Took 0.146 seconds to find inactive users,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,Found 4000 inactive users,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive,0.146312,4000.0,5000.0,1000.0,user_list_test
86,Took 0.093 seconds to find ready users,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,Found 1000 ready users,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ready,0.092930,1000.0,5000.0,1000.0,user_list_test


Parse one `.jsonl` file to:

1. select only test results
2. drop empty and irrelevant columns
3. restore integer types for user counts (missing values forces pandas to use float)
4. cast time to integer milliseconds

In [3]:
def load_one_run(path):
    """Load one .jsonl log file, filtering only our relevant test results"""
    df = pd.read_json(path, lines=True)
    # report time in ms
    df["time"] *= 1000
    # select only user_list_test rows, and drop uninteresting message/test columns
    return (
        df[df.test == "user_list_test"]  # select our user_list_test results
        .dropna(axis=1)  # drop empty columns
        .drop(columns=["message", "test"])  # drop uninteresting columns
        .astype(
            {"found": int, "total": int, "active": int, "time": int}
        )  # cast floats to ints now that we have no NaNs
        .reindex()
    )


one = load_one_run(path)
one.head()

,state,time,found,total,active
9,all,219,5000,5000,1000
11,active,80,1000,5000,1000
13,inactive,150,4000,5000,1000
15,ready,92,1000,5000,1000
17,all,514,5000,5000,1000


Our data is stored in one directory of runs per jupyterhub configuration.
We have two variables and two values for each variable:

- `version=before` means jupyterhub 1.2.1
- `version=after` means jupyterhub master with [pr3177](https://github.com/jupyterhub/jupyterhub/pull/3177) merged
- `db` is either `sqlite` or `postgres` identifying the database backend


In [4]:
import glob
def load_set(name):
    """Load one collection of samples for a given configuration"""
    return pd.concat([load_one_run(path) for path in glob.glob(f"runs/{name}/*.jsonl")], ignore_index=True)
load_set("before-sqlite")

,state,time,found,total,active
0,all,500,10000,10000,0
1,active,302,10000,10000,0
2,inactive,274,10000,10000,0
3,ready,273,10000,10000,0
4,all,439,10000,10000,0
...,...,...,...,...,...
2435,ready,286,10000,10000,500
2436,all,286,10000,10000,500
2437,active,510,10000,10000,500
2438,inactive,284,10000,10000,500


In [5]:
os.listdir("runs")

['after-postgres', 'after-sqlite', 'before-postgres', 'before-sqlite']

In [6]:
def load_everything():
    """Load all runs, adding a 'run' column identifying each run"""
    everything = {}
    dfs = []
    for run in os.listdir("runs"):
        # run has format before-sqlite, after-postgres
        version, db = run.split("-")
        df = load_set(run)
        df["run"] = run
        df["version"] = version
        df["db"] = db
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)


df = everything = load_everything()
df

,state,time,found,total,active,run,version,db
0,all,292,10000,10000,0,after-postgres,after,postgres
1,active,11,0,10000,0,after-postgres,after,postgres
2,inactive,476,10000,10000,0,after-postgres,after,postgres
3,ready,11,0,10000,0,after-postgres,after,postgres
4,all,276,10000,10000,0,after-postgres,after,postgres
...,...,...,...,...,...,...,...,...
9755,ready,286,10000,10000,500,before-sqlite,before,sqlite
9756,all,286,10000,10000,500,before-sqlite,before,sqlite
9757,active,510,10000,10000,500,before-sqlite,before,sqlite
9758,inactive,284,10000,10000,500,before-sqlite,before,sqlite


Now we have one dataframe collecting all of our samples:

- `time` is the duration of the request in milliseconds. This does not include client-side JSON parsing time.
- `state` is the `?state=` argument:
  - `all` (actually means no param is given) means return all users (only behavior available in `before`)
  - `active` selects only active users (i.e. those with running servers)
  - `ready` selects active users and then adds a secondary filter on the only-in-memory `ready` state.
    This will always return the same number of users as `active`,
    a worst-case-scenario for the cost of applying the filter.
  - `inactive` is the complement of `active`
- `total` is the total number of users registered in juptyerhub. This was run for a range of 0 - 20000 users. This 
- `active` is the number of active users with running 
- `found` is the number of users returned by the query:
  - `found = total` for all `before` cases
  - `found = active` for `after` where `state=active` or `state=ready`
  - `found = total - active` for `after` where `state=inactive`

Each combination of `run`, `total`, `active` is samples 10 times, since these can be noisy

## Plotting

The main concerns in the PR are:

1. is there a cost to doing this for getting all users?
2. how much savings are there when we know we only want a small subset of users?
3. when performing a more complex query returing a large number of users, is it worse than retrieving all users?

Let's get started.


### Existing behavior

First, plot the existing behavior (Get all users) for all configurations.
This shows a rather wide variance in performance, but no major cost.

Strangely, `after-sqlite` seems to have the widest distribution in runtime. This was peculiar and I can't explain it.


In [7]:
run_selection = alt.selection_multi(fields=["run"], bind="legend")
run_selected_opacity = alt.condition(run_selection, alt.value(1), alt.value(0.2))
run_selected_area_opacity = alt.condition(run_selection, alt.value(0.3), alt.value(0))

In [8]:
subset = df[(df["state"] == "all") & (df["total"] > 0)]

points = (
    alt.Chart(subset, title="Get all users")
    .mark_point()
    .encode(
        alt.X("total", scale=alt.Scale(type="log")),
        alt.Y("time", scale=alt.Scale(type="log")),
        color="run",
        tooltip=["run", "total", "time", "state"],
        opacity=run_selected_opacity,
    )
).add_selection(run_selection)

band = (
    alt.Chart(subset)
    .mark_errorband(extent="ci")
    .encode(
        alt.X("total", scale=alt.Scale(type="log")),
        alt.Y("time", scale=alt.Scale(type="log")),
        color="run",
        tooltip=["run"],
        opacity=run_selected_area_opacity,
    )
)

(points + band).interactive()

alt.LayerChart(...)

### Get just active users

Here, we make a chart for each of our total number of users,
plotting request time to fetch `?state=active` versus the number of active users.

Since the `before` samples don't actually do any server-side filtering,
they have the same $O(total)$ behavior, regardless of the number of active users,
aside from possible load due to the active spawners in the background.

In [9]:
charts = []
for n in sorted(df["total"].unique()):
    if n == 0:
        continue
    subset = df[(df["total"] == n) & (df["state"] == "active")]

    points = (
        alt.Chart(subset, title=f"{n} users, retrieve active")
        .mark_point()
        .encode(
            alt.X("active"),
            alt.Y("time", scale=alt.Scale(type="log")),
            color="run",
            tooltip=["run", "active", "time", "state"],
            opacity=run_selected_opacity,
        )
    ).add_selection(run_selection)

    band = (
        alt.Chart(subset)
        .mark_errorband(extent="ci")
        .encode(
            alt.X("active"),
            alt.Y("time", scale=alt.Scale(type="log")),
            color="run",
            tooltip=["run"],
            opacity=run_selected_area_opacity,
        )
    )

    charts.append((points + band).interactive())

alt.vconcat(*charts).resolve_scale(y="shared")

alt.VConcatChart(...)

So we can see for 20k users, where fetching all users takes ~1 second,
fetching 100 active users still takes only 20ms. 50x improvement!

We can plot the same information, this time selecting 100 active users
and plotting time versus the total number of users.

This lets us directly compare the scaling costs of the behavior in jupyterhub 1.2.1
with what should be fairly flat performacne with the proposed filter.

In [10]:
subset = df[(df["active"] == 100) & (df["state"] == "active") & (df["total"] > 0)]

points = (
    alt.Chart(subset, title="Retrieve 100 active users")
    .mark_point()
    .encode(
        alt.X("total", scale=alt.Scale(type="log")),
        alt.Y("time", scale=alt.Scale(type="log")),
        color="run",
        tooltip=["run", "total", "time", "state"],
        opacity=run_selected_opacity,
    )
).add_selection(run_selection)

band = (
    alt.Chart(subset)
    .mark_errorband(extent="ci")
    .encode(
        alt.X("total", scale=alt.Scale(type="log")),
        alt.Y("time", scale=alt.Scale(type="log")),
        color="run",
        tooltip=["run"],
        opacity=run_selected_area_opacity,
    )
)

(points + band).interactive()

alt.LayerChart(...)

### Fetching inactive users

`inactive` is the complementary query to `active`.
When there is a large number of inactive users,
it could be that the inactive query is significantly more expensive than the simpler
"get everything" query to fetch all users.
We don't seem to incur any appreciable penalty for the more complex query with a large number of results.

In [11]:
charts = []

for total in (5000, 10000, 20000):
    subset = df[(df["total"] == total) & (df["state"] == "inactive")]
    
    points = (
        alt.Chart(subset, title=f"{total} users, retrieve inactive")
        .mark_point()
        .encode(
            alt.X("active"),
            alt.Y("time", scale=alt.Scale(type="log")),
            color="run",
            tooltip=["run", "active", "time", "state"],
            opacity=run_selected_opacity,
        )
    ).add_selection(run_selection)

    band = (
        alt.Chart(subset)
        .mark_errorband(extent="ci")
        .encode(
            alt.X("active"),
            alt.Y("time", scale=alt.Scale(type="log")),
            color="run",
            tooltip=["run"],
            opacity=run_selected_area_opacity,
        )
    )

    charts.append((points + band).interactive())

alt.vconcat(*charts)

alt.VConcatChart(...)

### Comparing queries

Finally, we can plot the timings for each of our queries
when there are 100 active users.

In this chart, we collect all `before` measurements into a single line to compare with
our new queries, to see how they scale with the total number of users.

We should expect flat time for `ready/active` and `inactive` should scale similar to `all`,
being about the same as `total` gets >> 100.

In [14]:
subset = df[(df["active"] == 100) & (df["db"] == "postgres") & (df["total"] > 0)].copy()
# fake 'state=before' for all `before-postgres` runs, which are really the same
# this lets us get 5 lines on one chart, comparing 4 `after-` queries with before behavior
subset.loc[subset["version"] == "before", "state"] = "jupyterhub-1.2.1"

selection = alt.selection_multi(fields=["state"], bind="legend")
points = (
    alt.Chart(subset, title=f"100 active users (postgres)")
    .mark_point()
    .encode(
        alt.X("total", scale=alt.Scale(type="log")),
        alt.Y("time", scale=alt.Scale(type="log")),
        color="state",
        tooltip=["run", "total", "time", "state"],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    )
).add_selection(selection)

band = (
    alt.Chart(subset)
    .mark_errorband(extent="ci")
    .encode(
        alt.X("total", scale=alt.Scale(type="log")),
        alt.Y("time", scale=alt.Scale(type="log")),
        color="state",
        opacity=alt.condition(selection, alt.value(0.5), alt.value(0.1)),
    )
)
(points + band)

alt.LayerChart(...)